In [1]:
import json
import os
from torch.utils.data.dataset import random_split


In [2]:
print(os.listdir('./'))

['test.py', 'data_load.py', 'model.py', 'train.py', 'data_handle.ipynb', 'checkpoints', 'temp.txt', '__pycache__', '.ipynb_checkpoints']


In [4]:
with open('../data/train.json','rt',encoding='utf-8') as fin1,open('../data/test.json','rt',encoding='utf-8') as fin2,\
open('../data/train.txt','w',encoding='utf-8') as fout1,open('../data/valid.txt','w',encoding='utf-8') as fout2,open('../data/test.txt','w',encoding='utf-8') as fout3:
    labels=set()
    train_=[]
    for line in fin1:
        line = line.strip()
        if not line:
            continue  #结束则跳出循环
        sentence = json.loads(line)#将json数据转换成python对象  这里是转换为了字典
        
        train_.append(sentence)
    train_size=len(train_)
    val_size = int(0.25 * len(train_))
    train_,val_= random_split(train_, [train_size-val_size,val_size])
    train_=list(train_)
    val_=list(val_)

    for tr in train_:
        sentence_text=list(tr["sentText"].strip().strip('"'))
#         print(len(sentence_text))
        label=['O' for i in range(len(sentence_text))]
#         print(sentence_text[112])
        for e in tr['entityMentions']:
            labels.add("B-"+e['label'])
            labels.add("I-"+e['label'])
            label[e['start']]='B-'+e['label']
            for i in range(e['start']+1,e['end']):
                label[i]='I-'+e['label']
        for j in range(len(label)):
            fout1.write(sentence_text[j]+'\t'+label[j]+'\n')
        fout1.write('\n')
#             print(sentence_text[j],label[j])
        #break
    
    for vl in val_:
        sentence_text=list(vl["sentText"].strip().strip('"'))
#         print(len(sentence_text))
        label=['O' for i in range(len(sentence_text))]
#         print(sentence_text[112])
        for e in vl['entityMentions']:
            labels.add("B-"+e['label'])
            labels.add("I-"+e['label'])
            label[e['start']]='B-'+e['label']
            for i in range(e['start']+1,e['end']):
                label[i]='I-'+e['label']
        for j in range(len(label)):
            fout2.write(sentence_text[j]+'\t'+label[j]+'\n')
        fout2.write('\n')
    
    
    
    
    test_=[]
    for line in fin2:
        line = line.strip()
        if not line:
            continue  #结束则跳出循环
        sentence = json.loads(line)#将json数据转换成python对象  这里是转换为了字典
        test_.append(sentence)
    
    for te in test_:
        sentence_text=list(te["sentText"].strip().strip('"'))
#         print(len(sentence_text))
        label=['O' for i in range(len(sentence_text))]
#         print(sentence_text[112])
        for e in te['entityMentions']:
            labels.add("B-"+e['label'])
            labels.add("I-"+e['label'])
            label[e['start']]='B-'+e['label']
            for i in range(e['start']+1,e['end']):
                label[i]='I-'+e['label']
        for j in range(len(label)):
            fout3.write(sentence_text[j]+'\t'+label[j]+'\n')
        fout3.write('\n')
    

In [56]:
labels.add('O')
labels.add('<PAD>')
tuple(labels)

('O',
 'I-Ns',
 'I-NDR',
 'I-NW',
 'B-Nh',
 '<pad>',
 'B-NT',
 'B-NW',
 'B-Ns',
 'B-NDR',
 'I-NT',
 'I-Nh')

In [17]:
entries = open('conll2003/test.txt', 'r').read().strip().split("\n\n")
sents, tags_li = [], [] # list of lists
for entry in entries:
    words = [line.split()[0] for line in entry.splitlines()]

In [62]:
from my_data_load import pad,NerDataset


In [80]:
batch = NerDataset("data2/valid.txt")
maxl=0

f = lambda x: [sample[x] for sample in batch]
for l in f(-1):
    l=max(l,maxl)
print(l)

140


In [81]:
l=[1] + [0]*(20 - 1)

In [82]:
l

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [86]:
with open('data2/test_inf.txt','rt',encoding='utf-8') as fin1,open('data2/my_test_inf.txt','w',encoding='utf-8') as fout1:
    
    train_=[]
    for line in fin1:
        line = line.strip()
        if not line:
            continue  #结束则跳出循环    
        train_.append(line)
    
    for tr in train_:
        sentence_text=list(tr.strip().strip('"'))
#         print(len(sentence_text))
        label=['O' for i in range(len(sentence_text))]
#         print(sentence_text[112])
        
        for j in range(len(label)):
            fout1.write(sentence_text[j]+'\t'+label[j]+'\n')
        fout1.write('\n')

In [99]:
lmax=0
entries = open('data2/train.txt', 'r',encoding='utf-8').read().strip().split("\n\n")
for entry in entries:
    lmax=max(lmax,len(entry.splitlines()))
    
    

In [100]:
lmax

486

In [1]:
import csv
import re
###创建pipeline中关系抽取测试集
path='../data/ner_result.tsv'
with open(path, 'w') as fout:
    result = open("temp.txt", "r").read().strip().split("\n\n")
    
    wtest=csv.writer(fout,delimiter='\t')
    for ID,entry in enumerate(result):
        entities=[]
        entity={}
        words = [line.split()[0] for line in entry.splitlines()]
        tags = ([line.split()[-1] for line in entry.splitlines()])
#         print(words,tags)
        for idx,t in enumerate(tags):
#             print(t)
            if t=='O':
                if entity!={}:
                    entity['end']=idx
                    entities.append(entity)
                    entity={}   #type start end text
                continue
            elif t.startswith('B'): 
                entity['start']=idx
                entity['text']=words[idx]
                entity['type']=t.split('-')[1]
            elif t.startswith('I'):
                if'text' in entity:
                    entity['text']+=words[idx]
            
        entitypair_list=[]
        for e1 in entities:
            if e1['type']!='Nh' and e1['type']!='NDR':
                continue
            for e2 in entities:
                if e2['type']!='Nh' and e2['type']!='NDR':
                    continue
                if e1!=e2 and e1['text']!=e2['text']:
                    entitypair=[]
                    entitypair.append(e1)
                    entitypair.append(e2)
                    if entitypair not in entitypair_list:
                        entitypair_list.append(entitypair)
        
        for sid,ep in enumerate(entitypair_list):
            sent=words.copy()
            e1start=ep[0]['start']
            e1end=ep[0]['end']
            e2start=ep[1]['start']
            e2end=ep[1]['end']
            
            if e1end>400 or e2end>400:
                continue
                
            if e1start<e2start:
                sent.insert(e2end,'[E22]')
                sent.insert(e2start,'[E21]')
                sent.insert(e1end,'[E12]')
                sent.insert(e1start,'[E11]')
            else:
                sent.insert(e1end,'[E12]')
                sent.insert(e1start,'[E11]')
                sent.insert(e2end,'[E22]')
                sent.insert(e2start,'[E21]')
                
            sent=''.join(sent)
#             sent_sub=re.sub(u'((?:经审理查明|[\u4e00-\u9fa5]*指控)(?:，|：))','',sent)    
            wtest.writerow((sid,sent,0,ID))
#         print(entitypair_list,len(entitypair_list))
#         print(words)
#         print(tags)
#         print(entities)
      
            